In [1]:
def TA(df):  
    import talib
    #報酬率
    #1日
    df["1_returns"]=((df["Close"])/(df["Close"].shift())-1)*100
    #5日
    df["5_returns"]=((df["Close"])/(df["Close"].shift(periods=5))-1)*100
    #20日
    df["20_returns"]=((df["Close"])/(df["Close"].shift(periods=20))-1)*100

    #動能指標MTM
    #7
    df["7momentum"] = df["Close"] - df["Close"].shift(periods=7)
    #14
    df["14momentum"] = df["Close"] - df["Close"].shift(periods=14)
    #28
    df["28momentum"] = df["Close"] - df["Close"].shift(periods=28)

    #簡單移動均線MA
    #5
    df["5ma"] = df["Close"].rolling(window =5).mean()
    #10
    df["10ma"] = df["Close"].rolling(window =10).mean()
    #20
    df["20ma"] = df["Close"].rolling(window =20).mean()

    #指數移動均線EMA
    #5
    k = 2.0/(5+1)
    df['5ema'] = df['Close']
    df['5ema'] = (df['Close'] - df['5ema'].shift())*k + df['5ema'].shift()
    #10
    k = 2.0/(10+1)
    df['10ema'] = df['Close']
    df['10ema'] = (df['Close'] - df['10ema'].shift())*k + df['10ema'].shift()
    #20
    k = 2.0/(20+1)
    df['20ema'] = df['Close']
    df['20ema'] = (df['Close'] - df['20ema'].shift())*k + df['20ema'].shift()

    #價格變動率RC
    #5
    df['5rc'] = (df['Close']-df["Close"].shift(5)) / (df['Close'].shift(5))
    #10
    df['10rc'] = (df['Close']-df["Close"].shift(10)) / (df['Close'].shift(10))
    #20
    df['20r '] = (df['Close']-df["Close"].shift(20)) / (df['Close'].shift(20))

    #布林通道
    #布林通道上軌
    df["10ma"] = df["Close"].rolling(window =10).mean()
    df["bollinger_upper"]= 2*df['Close'].rolling(window =10).std() + df['10ma']
    #布林通道下軌
    df["bollinger_lower"]= df['10ma']-2*df['Close'].rolling(window =10).std()

    #KD指標
    #RSV
    df["RSV"]=((df["Close"]-df["Low"].rolling(window=9).min())/(df["High"].rolling(window=9).max()-df["Low"].rolling(window=9).min()))*100
    fastk,fastd=talib.STOCH(df["High"],df["Low"],df["Close"],9,3)
    #K
    df["K"]=fastk
    df["D"]=fastd

    #MACD
    df['ema_M'] = df['Close']
    df['ema_M'] = (df['Close'] - df['ema_M'].shift())*0.1528 + df['ema_M'].shift()
    df['ema_L'] = df['Close']
    df['ema_L'] = (df['Close'] - df['ema_L'].shift())*0.0741 + df['ema_L'].shift()
    df["DIF"]=df["ema_L"]-df["ema_M"]
    df.drop(columns=["ema_M","ema_L"],axis=1,inplace=True)

    #Pivot Point
    #典型
    df_temp = df.shift()
    df["classic_pp"]=(df_temp["High"]+df_temp["Low"]+df_temp["Close"])/3
    #woodie
    df['woodie_pp'] = ( df_temp['High'] + df_temp['Low'] + df_temp['Close']*2)/4.0

    # 14日ATR
    df['14atr'] = df['High'] - df['Low']
    df['14atr'] = (df["14atr"].rolling(14).sum())/14

     #RSI指標
    df['shift']=df['Close'].shift()
    df['gain']=df['Close']-df['shift']
    df.loc[df['gain'] < 0,'gain']=0
    df['loss']=df['shift']-df['Close']
    df.loc[df['loss'] <= 0,'loss']=0
    df["6_rsi"]=(df["gain"].rolling(6).sum())/(df["gain"].rolling(6).sum()+df["loss"].rolling(6).sum())*100
    df["12_rsi"]=(df["gain"].rolling(12).sum())/(df["gain"].rolling(12).sum()+df["loss"].rolling(12).sum())*100
    df.drop(['shift','gain','loss',],axis=1,inplace=True)

    #W%R 威廉指標
    #14
    df["14_wr"]=((df["High"].rolling(window=14).max()-df["Close"])/((df["High"].rolling(window=14).max())-(df["Low"].rolling(window=14).min())))*100

    #7日黃金分割線
    high = df['Close'].rolling(window=7).max()
    low = df['Close'].rolling(window=7).min()
    df['uptrend_7day_fibo_0.382'] = high - ((high-low)*0.382)
    df['downtrend_7day_fibo_0.382'] = low + ((high-low)*0.382)
    df['uptrend_7day_fibo_0.5'] = high - ((high-low)*0.5)
    df['downtrend_7day_fibo_0.5'] = low + ((high-low)*0.5)
    df['uptrend_7day_fibo_0.618'] = high - ((high-low)*0.618)
    df['downtrend_7day_fibo_0.618'] = low + ((high-low)*0.618)
    #28日黃金分割線
    high = df['Close'].rolling(window=28).max()
    low = df['Close'].rolling(window=28).min()
    df['uptrend_28day_fibo_0.382'] = high - ((high-low)*0.382)
    df['downtrend_28day_fibo_0.382'] = low + ((high-low)*0.382)
    df['uptrend_28day_fibo_0.5'] = high - ((high-low)*0.5)
    df['downtrend_28day_fibo_0.5'] = low + ((high-low)*0.5)
    df['uptrend_28day_fibo_0.618'] = high - ((high-low)*0.618)
    df['downtrend_28day_fibo_0.618'] = low + ((high-low)*0.618)

    #BIAS
    #5、10、20
    df["5_bias"]=((df["Close"]-df["5ma"])/df["5ma"])*100
    df["10_bias"]=((df["Close"]-df["10ma"])/df["10ma"])*100
    df["20_bias"]=((df["Close"]-df["20ma"])/df["20ma"])*100

     #PSY心理線
    df["up"]=df["Close"]-df["Close"].shift()
    df.loc[df["up"]>0,"up"]=1
    df.loc[df["up"]<=0,"up"]=0
    df["psy"]=(df["up"].rolling(window=10).sum()/10)*100
    df.drop(["up"],axis=1,inplace=True)

    #OBV指標
    df['positive_signed_volume'] = df[df['1_returns'] >0]['Volume']
    df['positive_signed_volume'].replace(float('nan'),0,inplace=True)
    df['negative_signed_volume'] = df[df['1_returns'] < 0]['Volume']*(-1.0)
    df['negative_signed_volume'].replace(float('nan'),0,inplace=True)
    df['signed_volume'] = df['positive_signed_volume'] + df['negative_signed_volume']
    df['5_obv'] = (df['signed_volume'].rolling(window=5).sum())
    df['10_obv'] = (df['signed_volume'].rolling(window=10).sum())
    df['20_obv'] = (df['signed_volume'].rolling(window=20).sum())

    #VR
    df['negative_signed_volume']=df['negative_signed_volume']*(-1.0)
    df["vr"]=((df["positive_signed_volume"].rolling(10).sum()+0.5*df["Volume"].rolling(10).sum())/(df["negative_signed_volume"].rolling(10).sum()+0.5*df["Volume"].rolling(10).sum()))*100
    df.drop(['signed_volume','positive_signed_volume',"negative_signed_volume"],axis = 1,inplace=True)

    #CCI
    df["TP"]=(df["High"]+df["Low"]+df["Close"])/3
    df["avgTP"]=(df["TP"].rolling(14).sum())/14
    df["MD"]=abs(df["TP"]-df["avgTP"])
    df["avgMD"]=((df["MD"].rolling(14).sum())/14)*0.015
    df["CCI"]=(df["TP"]-df["avgTP"])/df["avgMD"]
    df.drop(["TP","avgTP","MD","avgMD"],axis=1,inplace=True)
    
    #CDP
    df["cdp_AH"]=df["woodie_pp"]+(df_temp["High"]-df_temp["Low"])
    df["cdp_NH"]=df["woodie_pp"]*2-df_temp["Low"]
    df["cdp_AL"]=df["woodie_pp"]-(df_temp["High"]-df_temp["Low"])
    df["cdp_NL"]=df["woodie_pp"]*2-df_temp["High"]
    
    return df

def afterTA(df,days):
    #把na值刪掉
    df.dropna(inplace=True)
    #預測後幾天的收盤價
    Target=df["Close"].shift(-1*days)
    #清理na值
    Target.dropna(inplace=True)
    #df後幾天的資料不要
    df=df.drop(df.index[-1*days:])
    return Target,df

def afterTA(df,days):
    #把na值刪掉
    df.dropna(inplace=True)
    #預測後幾天的收盤價
    Target=df["Close"].shift(-1*days)
    #清理na值
    Target.dropna(inplace=True)
    #df後幾天的資料不要
    df=df.drop(df.index[-1*days:])
    return Target,df

#將feature_df_transformed分成訓練跟測試集
def traintest(df,Target,train_ratio,Open):
    #分出訓練、測試期間
    #將訓練跟測試期分比例
    train_num=round(len(df)*train_ratio)
    #訓練期
    train_df=df[:train_num]
    train_target=Target[:train_num]
    #測試期
    test_df=df[train_num:]
    test_target=Target[train_num:]
    Open_target=Open[train_num:]
    Open_target.dropna(inplace=True)
    return train_df,train_target,test_df,test_target,Open_target

#批次標準化def
def batchnormalization(data):
    import tensorflow as tf
    w=tf.constant(data.tolist())
    mean,var=tf.nn.moments(w,[0])
    offset=0
    scale=1
    epsilon=1e-8
    a=tf.nn.batch_normalization(w,mean,var,offset,scale,epsilon,name=None)
    return a

#資料分集
def dataset_batch(train_df,train_target,splits,test_df,test_target):
    from sklearn.model_selection import TimeSeriesSplit
    import tensorflow as tf
    import numpy as np
    import warnings
    warnings.filterwarnings('ignore')
    X=np.array(train_df)
    y=np.array(train_target)
    #使用時間序列分割
    tscv=TimeSeriesSplit(n_splits=splits)
    X_train=[]
    X_validation=[]
    y_train=[]
    y_validation=[]
    for train_index, validation_index in tscv.split(X):
        X_train.append(pd.DataFrame(X[train_index]).apply(lambda x:((x-x.mean())/(x.max()-x.min()))))
        X_validation.append(pd.DataFrame(X[validation_index]).apply(lambda x:((x-x.mean())/(x.max()-x.min()))))
        y_train.append(pd.DataFrame(y[train_index]).apply(lambda x:((x-x.mean())/(x.max()-x.min()))))
        y_validation.append(pd.DataFrame(y[validation_index]).apply(lambda x:((x-x.mean())/(x.max()-x.min()))))


    #test_df跟整的train_df做批次標準化
    X_test=pd.DataFrame(test_df).apply(lambda x:((x-x.mean())/(x.max()-x.min())))
    y_test=pd.DataFrame(test_target).apply(lambda x:((x-x.mean())/(x.max()-x.min())))
    all_X_train=pd.DataFrame(train_df).apply(lambda x:((x-x.mean())/(x.max()-x.min())))
    all_y_train=pd.DataFrame(train_target).apply(lambda x:((x-x.mean())/(x.max()-x.min())))
    return X_train,X_validation,y_train,y_validation,X_test,y_test,all_X_train,all_y_train

#找出最佳參數
def bestscore(X_train,y_train,X_validation,y_validation,opt):
    from sklearn.svm import SVR
    #最佳化參數
    opt.fit(X_train,y_train)
    optscore=opt.score(X_validation, y_validation)
    optparams=opt.best_params_
    #比較組別
    #原始SVR
    svr=SVR()
    svr.fit(X_train,y_train)
    svrscore=svr.score(X_validation,y_validation)
    return optscore,optparams,svrscore

#找出最佳組合於clfs
#clftodf,最佳組合,最佳化最大值,最佳平均值
def clfs(optscore,optparams,svrscore):
    import pandas as pd
    #找出最大值的位置及組合
    clfs=pd.DataFrame()
    clfs["OPT"]=optscore
    clfs["SVR"]=svrscore
    clfs["n_iter"]=[1,2,3,4,5]
    #最佳參數組合
    optmax=optparams[clfs["OPT"].idxmax()]
    return clfs,optmax,clfs["OPT"].max(),clfs["OPT"].mean()

#計算交易策略
def resdf(startday,endday,sillnum,days,test_target,y_test,testsvr,X_test,current,Open_target,testtime):
    import pandas as pd
    from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score
    import numpy as np
    import tensorflow as tf
    import warnings
    warnings.filterwarnings('ignore')
    temp=pd.DataFrame()
    temp["time"]=test_target.index
    #當天實際收盤價_正規化
    temp["today_c_nor"]=list(pd.DataFrame(current).apply(lambda x:((x-x.mean())/(x.max()-x.min())))["Close"])
    #SVR預測值
    temp["pre_c_nor"]=testsvr.predict(X_test)
    #當天實際收盤價
    temp["today_c"]=list(current)
    #實際未來開盤價
    temp["tomorrow_o"]=list(Open_target)
    #實際未來收盤價
    temp["tomorrow_c"]=list(test_target)
    #實際未來收盤價_正規化
    temp["tomorrow_c_nor"]=list(y_test["Close"])
    #分期間(整個,平穩，動盪)
    temp=temp[temp["time"].between(startday,endday)]

    cost=(0.1425*0.6+0.1425*0.6+0.3*0.5)/100
    a=temp[temp["pre_c_nor"]>temp["today_c_nor"]]
    a=a[a["tomorrow_o"]<a["today_c"]*(1-cost-sillnum)]
    a["profit_cost"]=((a["tomorrow_c"]-a["tomorrow_o"])/a["tomorrow_o"])-cost
    a["profit"]=((a["tomorrow_c"]-a["tomorrow_o"])/a["tomorrow_o"])

    resdf_out=pd.DataFrame(index=[testtime])
    resdf_out["年化報酬率_有"]=a.profit_cost.sum()*250/len(temp)
    resdf_out["年化報酬率_無"]=a.profit.sum()*250/len(temp)
    if len(a)>0:
        resdf_out["年化報酬率_有_勝率"]=len(a[a["profit_cost"]>0])/len(a)
        resdf_out["年化報酬率_無_勝率"]=len(a[a["profit"]>0])/len(a)
    else:
        resdf_out["年化報酬率_有_勝率"]="無"
        resdf_out["年化報酬率_無_勝率"]="無"
    resdf_out["交易次數"]=len(a)
    temp=temp.reset_index()
    resdf_out["BandH"]=((temp["tomorrow_c"][len(temp)-1]-temp["tomorrow_c"][0])/temp["tomorrow_c"][0])    
    resdf_out["r_square"]=round(r2_score(temp["tomorrow_c_nor"],temp["pre_c_nor"]),4)
    resdf_out["MAE"]=round(mean_absolute_error(temp["tomorrow_c_nor"],temp["pre_c_nor"]),4)
    resdf_out["MSE"]=round(mean_squared_error(temp["tomorrow_c_nor"],temp["pre_c_nor"]),4)
    return resdf_out,temp

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
from skopt import BayesSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
import pandas as pd
import tensorflow as tf
import numpy as np
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import FactorAnalysis
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error,r2_score
import time
from sklearn.cross_decomposition import CCA
#從38公司裡取檔到pathlist
pathlist=[]
for dirPath, dirNames, fileNames in os.walk(r"C:\Users\ina\Desktop\excel"):
    for f in fileNames:
        pathlist.append(os.path.join(dirPath, f))
#預測天數
days=1
#時間序列分割
splits=5
#超參數優化迭代次數
n_iter=10
#超參數優化模型和參數設置
opt=BayesSearchCV(
        SVR(),
        {
        "kernel":("linear","rbf","sigmoid","poly"),
        "degree":(1,10),
        "gamma":(1e-9,1.0,"log-uniform"),
        "C":(1,10)
        },
        scoring="r2",
        n_iter=n_iter,
        cv=TimeSeriesSplit(n_splits=5),
        refit=True,
        return_train_score=True
)
#模型名稱
modelname="FA_CCA_SVR"
#交易策略門檻值
sillnum=0.01

In [ ]:
for excel in pathlist:
    df=pd.read_excel(excel,index_col=0)
    #將日期設置為index
    companyname=excel[27:-5]
    print(companyname)
    #開始日期
    start="2015-01-01"
    #結束日期
    df=df[df.index>start]
    #分成技術面跟籌碼面
    ta_df=df[df.columns[0:5]]
    chip_df=df[df.columns[5:]]
    chip_df["Close"]=df["Close"]
    
    #def
    #匯入技術指標
    ta_data=TA(ta_df)

    #將ta_data分成Target跟feature_df
    #分別將ta_df和chip_df匯入
    #使用afterTA把空值刪除掉
    ta_Target,ta_feature_df=afterTA(ta_data,days)
    chip_Target,chip_feature_df=afterTA(chip_df,days)
    
    #取出當天收盤價
    current=ta_feature_df["Close"][round(len(ta_feature_df)*0.8):]
    
    #取出隔一天開盤價
    openprice=ta_data.dropna()["Open"].shift(-1*days)
    
    #把chip_target和chip_feature_df長度縮短跟ta長度一樣
    diffdate=list(set(chip_Target.keys()).difference(set(ta_Target.keys())))
    chip_feature_df=chip_feature_df.drop(diffdate)
    chip_Target=chip_Target.drop(diffdate)

    #找出ta_feature_df的相關性
    a=ta_feature_df.corrwith(ta_Target,method="spearman")
    #由小到大排序相關性排行
    a=list(a.sort_values().index)
    #要選擇因素數量(Kaiser準則)
    ta_fa=FactorAnalyzer(rotation=None)
    #for 迴圈測試沒有奇異矩陣
    ta_dfnum=[j for j in range(0,len(a))]
    for i in range(0,len(a)):
        test=ta_feature_df.drop(columns=a[:i],axis=1)
        try:
            ta_fa.fit(test)
        except np.linalg.LinAlgError as err:
            if  "Singular matrix" in str(err):
                ta_dfnum.remove(i)
    ta_feature_df.drop(columns=a[:ta_dfnum[0]],axis=1,inplace=True)
    chip_feature_df.drop("Close",axis=1,inplace=True) 


    #找出chip_feature_df的相關性
    b=chip_feature_df.corrwith(ta_Target,method="spearman")
    #由小到大排序相關性排行
    b=list(b.sort_values().index)
    #要選擇因素數量(Kaiser準則)
    chip_fa=FactorAnalyzer(rotation=None)
    #for 迴圈測試沒有奇異矩陣
    chip_dfnum=[j for j in range(0,len(b))]
    for i in range(0,len(b)):
        test=chip_feature_df.drop(columns=b[:i],axis=1)
        try:
            chip_fa.fit(test)
        except np.linalg.LinAlgError as err:
            if  "Singular matrix" in str(err):
                chip_dfnum.remove(i)
    chip_feature_df.drop(columns=b[:chip_dfnum[0]],axis=1,inplace=True)
    
    #ta_FA
    ta_FA=FactorAnalysis()
    #將FA投射到新樣本
    #並將array轉為df以利訓練跟測試分集
    ta_feature_df_transformed=pd.DataFrame(ta_FA.fit_transform(ta_feature_df))
    #chip_FA
    chip_FA=FactorAnalysis()
    #將FA投射到新樣本
    #並將array轉為df以利訓練跟測試分集
    chip_feature_df_transformed=pd.DataFrame(chip_FA.fit_transform(chip_feature_df))

    #合併特徵檔
    two_df=pd.concat([ta_feature_df_transformed,chip_feature_df_transformed],axis=1,ignore_index=True,)

    #CCA特徵融合
    cca=CCA(n_components=1)
    #X值為合併特徵檔，Y值為這段期間收盤價
    cca.fit(two_df,chip_Target)
    cca_two_df,cca_Target=cca.transform(two_df,ta_Target)

    # 資料分集和批次標準化
    # 先分出訓練和測試期，再將訓練期分成訓練和驗證期，在逐一批次標準化
    train_df,train_target,test_df,test_target,Open_target=traintest(cca_two_df,ta_Target,0.8,openprice)
    X_train,X_validation,y_train,y_validation,X_test,y_test,all_X_train,all_y_train=dataset_batch(train_df,train_target,splits,test_df,test_target)
    
    #找出最佳參數
    optscorelist=[]
    optparamslist=[]
    svrscorelist=[]
    DRscorelist=[]
    #將timeseriessplit次數導入超參數優化模型
    for i in range(0,len(X_train)):
        optscore,optparams,svrscore=bestscore(X_train[i],y_train[i],X_validation[i],y_validation[i],opt)
        #超參數最佳化數值
        optscorelist.append(optscore)
        #超參數最佳化組合
        optparamslist.append(optparams)
        #普通SVR數值
        svrscorelist.append(svrscore)

    #找出最佳組合於clfs
    #clftodf,最佳組合,最佳化最大值,最佳平均值
    clfs_df,optmax,paramsmax,paramsmean=clfs(optscorelist,optparamslist,svrscorelist)

    #將最佳參數組合套用在測試期
    testsvr=SVR(C=optmax.get("C"),
    degree=optmax.get("degree"),
    gamma=optmax.get("gamma"),
    kernel=optmax.get("kernel")).fit(all_X_train,all_y_train)


    #不同時期計算損益
    entire,entire_detail=resdf(test_target.index[0],"2020-03-30",sillnum,days,test_target,y_test,testsvr,X_test,current,Open_target,"entire")
    safe,safe_detail=resdf(test_target.index[0],"2019-11-30",sillnum,days,test_target,y_test,testsvr,X_test,current,Open_target,"safe")
    bad,bad_detail=resdf("2019-12-01","2020-03-30",sillnum,days,test_target,y_test,testsvr,X_test,current,Open_target,"bad")

    #把所有結果顯示同張表
    concat_df=safe
    for a in [bad,entire]:
        concat_df=concat_df.append(a)
     
    print("整個時期有成本之年化報酬率",round(concat_df.iat[2,0]*100,2),"%")
    print("完成",companyname)